In [1]:
# Import necessary modules and functions
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import sys
sys.path.append('../src')  # add the directory containing your neural_nets module to the Python path
sys.path.append('../data')  # add the directory containing your data module to the Python path

from neural_nets import create_network_1, create_network_2, create_network_3, train_and_evaluate_nn, parameter_tuning_nn, median_and_iqr_nn
from data_generator import get_data, preprocess  # replace with your actual module and function names
import regression_functions as rf

In [ ]:
# Define a list of regression functions
regression_functions = [rf.m1, rf.m2, rf.m3, rf.m4, rf.m5, rf.m6, rf.m7, rf.m8]
neural_networks = [create_network_1, create_network_2, create_network_3]


# Define empty dataframe to store results
results = pd.DataFrame(columns=['regression_function', 'network', 'mse', 'iqr'])

# Loop through each regression function
for i, regression_func in enumerate(regression_functions):
    print(f"Regression Function {i+1}: {regression_func.__name__}")
    input_dim = regression_func.expected_dim
    x, y = get_data(regression_func, x_dim=input_dim, num_samples=1000, sigma=0.05)
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

    # Preprocess data
    train_data = preprocess(x_train, y_train, training=True)
    test_data = preprocess(x_test, y_test, training=False)
    
    # Tune network with one hidden layer
    units = [3, 6, 9, 12, 15]
    for j, network in enumerate(neural_networks):
        best_params = parameter_tuning_nn(network, units=units ,train_data=train_data, test_data=test_data, input_dim=input_dim)
        best_units = best_params['best_config']
        mse, iqr = median_and_iqr_nn(create_network_1, train_data, test_data, input_dim=input_dim, units=best_units)
        
        # Append the results to the dataframe
        results = results.append({
            'regression_function': regression_func.__name__,
            'network': network.__name__,
            'mse': mse,
            'iqr': iqr
        }, ignore_index=True)
        
        print(f"Best Units: {best_units}")
        print(f"MSE: {mse}")
        print(f"IQR: {iqr}")
    
    
    print("--------------------")
